# Swarm

Swarm は、エージェントが他のエージェントに能力に応じてタスクを委譲できるチームを実装しています。これは、OpenAI が実験プロジェクトで初めて導入したマルチエージェント設計パターンです。特別にセットされたツール（Function calling）を使用してエージェントが他のエージェントにタスクを委譲（Handoffs）できるようにすることです。これにより、エージェントは、SelectorGroupChat のような中央のオーケストレーターに頼らずに、エージェント遷移についてローカルで決定を下すことができます。

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/swarm.html

In [ ]:
#!pip install -U "autogen-agentchat"
#!pip install "autogen-ext[mcp,openai,azure]"

In [ ]:
import logging  
from typing import Any, List  
import os  
from dotenv import load_dotenv  

from autogen_agentchat.agents import AssistantAgent  
from autogen_agentchat.teams import Swarm  # keeps implementation simple & familiar  
from autogen_agentchat.conditions import TextMessageTermination, MaxMessageTermination, TextMentionTermination, TimeoutTermination

from autogen_core import CancellationToken  
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  
import warnings
warnings.simplefilter('ignore')

load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-openai

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_model = os.getenv("AZURE_OPENAI_MODEL")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools の定義(MCP over SSE)

In [ ]:

server_params = SseServerParams(  
    url=mcp_server_uri,  
    headers={"Content-Type": "application/json"},  
    timeout=30  
)  

# Fetch tools (async)  
tools = await mcp_server_tools(server_params)  
print(f"Number of Tools: {len(tools)}")
# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

## エージェント定義
### Handsoff パラメーター
Swarm の場合、AssistantAgent に handoffs 引数を設定して、どのエージェントにハンドオフ（委譲）できるかを指定することができます。Selector のような遷移先を決めるマネージャーが存在しないため、遷移先の決定は各エージェントのプロンプトに依存します

Handoff pydantic model を使用して、メッセージの内容とハンドオフの動作をカスタマイズすることもできます。

In [ ]:

# 3. -----------------  Agent Definitions -----------------  
coordinator = AssistantAgent(  
    name="coordinator",  
    model_client=model_client,  
    handoffs=["CRMBillingAgent", "ProductPromotionsAgent"],
    description="タスクを計画するエージェント。ユーザーのリクエストを適切な専門エージェントに振り分けてください。",
    system_message=(  
"""
あなたはコーディネーターです。ユーザーのリクエストを適切な専門エージェントに振り分けてください。
質問に直接答えてはいけません。必ず適切なエージェントにルーティングしてください。
- 請求やアカウント、CRM、SNS分析に関する質問は：CRMBillingAgent に振り分けること
- 製品やプロモーションに関する質問は：ProductPromotionsAgent に振り分けること
- 単純な質問に限り、直接回答しても構いません

# Rule
- Always send your coordinator first, then handoff to appropriate agent.
- Always handoff to a single agent at a time.
- 議論の結果を集約して最終的にユーザーに回答します。
- 自分は専門エージェントを実行することはできません
- handoff に失敗したら何度か再実行してください
- すべての handsoff が終了し、最終回答が完成したら文の最後に TERMINATE を含めること!

"""
)
)  

billing_agent = AssistantAgent(  
    name="CRMBillingAgent",  
    model_client=model_client,  
    description="CRM & 請求エージェントのエージェント。CRM／請求システムを照会する",
    tools=tools,  
    handoffs=["coordinator"],
    system_message=(
"""
あなたは「CRM & 請求エージェント（CRM & Billing Agent）」です。

- アカウント、サブスクリプション、請求書、支払い情報などを取得するために、
  構造化されたCRM／請求システムを照会します。
- 請求ポリシー、支払い処理、返金ルールなどに関する *ナレッジベース* 記事を確認し、
  回答が正確かつポリシーに準拠していることを保証します。
- 簡潔で構造化された情報を返答し、検出されたポリシー上の懸念事項があれば
  フラグを立てます。
- ツールを使って請求情報を確認してください。
- 請求に関係ない質問は coordinator に振り分けてください。
- Always handoff back to coordinator when analysis is complete.
"""
    ),  
)  

product_agent = AssistantAgent(  
    name="ProductPromotionsAgent",  
    model_client=model_client,  
    handoffs=["coordinator"],
    description="製品 & プロモーションエージェント。プロモーションのオファー、製品の在庫状況、適格条件、割引情報などを照会",
    tools=tools,  
    system_message=(  
"""
あなたは「製品 & プロモーションエージェント（Product & Promotions Agent）」です。

- 構造化された情報源から、プロモーションのオファー、製品の在庫状況、
  適格条件、割引情報などを取得します。
- *ナレッジベース* のFAQ、利用規約、ベストプラクティスを活用して、
  回答を補足します。
- 事実に基づいた、最新の製品／プロモーション情報を提供します。
- ツールを使って製品やプロモーションの詳細を確認してください。
- 製品に関係ない質問は coordinator に振り分けてください。
- Always handoff back to coordinator when analysis is complete.
"""
    ),  
)  


In [ ]:
# 4. -----------------  Assemble Team -----------------  
participants: List[AssistantAgent] = [  
    coordinator,  
    billing_agent,  
    product_agent
]  

## 停止条件
AutoGen には 無限ループを防止するため 8 つの組み込みの終了条件が定義されています。終了条件は以下のように OR 条件で指定できるのが便利です。



In [ ]:
# Define termination condition
max_msg_termination = MaxMessageTermination(max_messages=15)
text_termination = TextMentionTermination("TERMINATE")
time_terminarion = TimeoutTermination(120)
combined_termination = max_msg_termination | text_termination | time_terminarion


## Swarm の実行

In [ ]:
# 5. -----------------  Create Team Agent -----------------
team_agent = Swarm(
    participants=participants,
    termination_condition=combined_termination,
)

In [ ]:
with tracer.start_as_current_span("Swarm") as rollspan: # ルートスパンを作成
    task = "ユーザーID:123 の出荷状況を確認してください。"
    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Swarm") as rollspan: # ルートスパンを作成
    #task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "商品ID:339の商品詳細を教えて"
    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Swarm") as rollspan: # ルートスパンを作成
#task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "SNS分析を行い、日付ごとのツイート数を集計してください"
    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Swarm") as rollspan: # ルートスパンを作成
    task = "2024年9月~10月の受注数を集計し、最も受注数が多いかった日付を教えて"
    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Swarm") as rollspan: # ルートスパンを作成
    task = "2024年9月~10月の受注数を集計し、さらにSNS分析を行い、日付ごとのツイート数を集計した結果何かわかることはあるか？"
    await Console(team_agent.run_stream(task=task))